# Substitution model

## 1. Substitution model parameters

Let $\pi = (\pi_A,\pi_C,\pi_G,\pi_T)$ be the limiting probablities and $r=(\alpha,\beta,\gamma,\lambda,\rho,\omega)$ are six substitution rates. 


In [9]:
rm(list=ls())
pi = c(1/2,1/8,1/8,1/4)
rates = c(1,2,2,2,2,1)

ERROR: Error in parse(text = x, srcfile = src): <text>:2:24: unexpected input
1: #rm(list=ls())
2: pi = c(1/2,1/8,1/8,1/4)
                          ^


The rate matrix is given by 

$$Q = \begin{pmatrix}
- & \pi_C \alpha & \pi_G \beta & \pi_T \gamma \\
\pi_A \alpha & - & \pi_G \lambda & \pi_T \rho \\
\pi_A \beta  & \pi_C \lambda  & - & \pi_T \omega  \\
\pi_A \gamma & \pi_C \rho & \pi_G \omega & - 
\end{pmatrix}$$


In [10]:
qmatrix = matrix(0,4,4)
qmatrix[upper.tri(qmatrix)] = rates
qmatrix = qmatrix+t(qmatrix)
qmatrix = qmatrix %*% diag(pi)
for(i in 1:4) qmatrix[i,i] = -sum(qmatrix[i,-i])
qmatrix

ERROR: Error in parse(text = x, srcfile = src): <text>:1:24: unexpected input
1: qmatrix = matrix(0,4,4)
                           ^


In [14]:
x=1;
x;

ERROR: Error in parse(text = x, srcfile = src): <text>:1:5: unexpected input
1: x=1;
        ^


The diagonal values are those such that the row sums are equal to 0. Let $\delta= -\sum_{i=A,C,G,T} \pi_i Q_{ii}$. We transform the branch length $\tau=\delta t$ and $Q^*=Q/\delta$.

In [ ]:
delta = -sum(pi * diag(qmatrix))
delta

qmatrix = qmatrix/delta
qmatrix

t = 0.1
tau = delta*t
tau

[1] 1.125

-0.7777778,0.1111111,0.2222222,0.4444444
0.4444444,-1.1111111,0.2222222,0.4444444
0.8888889,0.2222222,-1.3333333,0.2222222
0.8888889,0.2222222,0.1111111,-1.2222222


[1] 0.1125

The transition probability matrix is given by $P(t)=e^{Q^*\tau}$.

In [ ]:
library(Matrix)
t=0.1
pmatrix = expm(qmatrix*tau)

pmatrix

4 x 4 Matrix of class "dgeMatrix"
           [,1]       [,2]       [,3]       [,4]
[1,] 0.91992610 0.01209794 0.02265866 0.04531731
[2,] 0.04839175 0.88363229 0.02265866 0.04531731
[3,] 0.09063462 0.02265866 0.86224630 0.02446042
[4,] 0.09063462 0.02265866 0.01223021 0.87447651

The probability distribution of 16 doublets $(AA,AC,...,TT)$ is equal to 

In [ ]:
doubleP=pmatrix
for(i in 1:4) doubleP[i,] = pi[i]*pmatrix[i,]
doubleP

4 x 4 Matrix of class "dgeMatrix"
            [,1]        [,2]        [,3]        [,4]
[1,] 0.459963048 0.006048968 0.011329328 0.022658656
[2,] 0.006048968 0.110454036 0.002832332 0.005664664
[3,] 0.011329328 0.002832332 0.107780788 0.003057553
[4,] 0.022658656 0.005664664 0.003057553 0.218619128

## 2. Simulation
2.1 For simplicity, the nucleotides (A,C,G,T) are coded (1,2,3,4).

In [ ]:
n=10000
x1=sample(1:4,n,prob=pi,replace=T)
x2=1:n
for(i in 1:n) x2[i] = sample(1:4,1,replace=T, prob=pmatrix[x1[i],])
data = cbind(x1,x2)
t(data)

x1,3,2,1,1,1,1,3,1,2,3,...,1,2,1,2,3,2,3,2,1,1
x2,3,2,1,1,1,1,3,1,2,3,...,1,2,1,2,3,1,3,2,1,1


## 3. Estimation
3.1 Estimating $\pi$

In [ ]:
pi_hat = 1:4
for(i in 1:4) pi_hat[i] = sum(x1==i)+sum(x2==i)
pi_hat = pi_hat/20000

print("The true pi")
pi

print("The estimates of pi")
pi_hat

[1] "The true pi"


[1] 0.500 0.125 0.125 0.250

[1] "The estimates of pi"


[1] 0.4941 0.1218 0.1308 0.2533

3.2 Estimating the probabilities of 16 doublet by their frequencies

In [ ]:
doublet = paste(x1,x2,sep="")
freq = table(doublet)
freq

doubleP_hat = matrix(freq,4,4)
doubleP_hat = doubleP_hat/sum(doubleP_hat)
doubleP_hat = (doubleP_hat+t(doubleP_hat))/2

print("the doubleP estimate")
doubleP_hat

print("the true doubleP")
doubleP

doublet
  11   12   13   14   21   22   23   24   31   32   33   34   41   42   43   44 
4714   57   64  110   62 1116   16   26   53   19 1199   42  108   24   24 2366 

[1] "the doubleP estimate"


0.47140,0.00595,0.00585,0.0109
0.00595,0.11160,0.00175,0.0025
0.00585,0.00175,0.11990,0.0033
0.01090,0.00250,0.00330,0.2366


[1] "the true doubleP"


4 x 4 Matrix of class "dgeMatrix"
            [,1]        [,2]        [,3]        [,4]
[1,] 0.476209355 0.005947661 0.005947661 0.011895323
[2,] 0.005947661 0.114591593 0.001486915 0.002973831
[3,] 0.005947661 0.001486915 0.114591593 0.002973831
[4,] 0.011895323 0.002973831 0.002973831 0.232157016

3.3 Estimating pmatrix

In [ ]:
pmatrix_hat=matrix(0,4,4)
for(i in 1:4) pmatrix_hat[i,] = doubleP_hat[i,]/pi_hat[i]

print("true pmatrix")
pmatrix

print("pmatrix estimate")
pmatrix_hat

[1] "true pmatrix"


4 x 4 Matrix of class "dgeMatrix"
           [,1]       [,2]       [,3]       [,4]
[1,] 0.95241871 0.01189532 0.01189532 0.02379065
[2,] 0.04758129 0.91673274 0.01189532 0.02379065
[3,] 0.04758129 0.01189532 0.91673274 0.02379065
[4,] 0.04758129 0.01189532 0.01189532 0.92862806

[1] "pmatrix estimate"


0.95405788,0.01204210,0.01183971,0.02206031
0.04885057,0.91625616,0.01436782,0.02052545
0.04472477,0.01337920,0.91666667,0.02522936
0.04303198,0.00986972,0.01302803,0.93407027


3.4 Estimating $Q\tau$

In [ ]:
eig = eigen(pmatrix_hat)
w = eig$vectors%*%diag(log(eig$values))%*%solve(eig$vectors)

print("the estimate is")
w

print("the true Qt is")
qmatrix*tau

[1] "the estimate is"


-0.04816717,0.01267673,0.01240932,0.02308112
0.05142505,-0.08801653,0.01520541,0.02138606
0.04687650,0.01415917,-0.08760660,0.02657093
0.04502321,0.01028355,0.01372079,-0.06902755


[1] "the true Qt is"


-0.05,0.0125,0.0125,0.025
0.05,-0.0875,0.0125,0.025
0.05,0.0125,-0.0875,0.025
0.05,0.0125,0.0125,-0.075


3.5 Estimating $\tau$

In [ ]:
tau_hat = -sum(pi_hat * diag(w))
tau_hat
tau

[1] 0.06346343

[1] 0.065625

3.6 Estimating qmatrix

In [ ]:
qmatrix_hat = w/tau_hat
for(i in 1:4) qmatrix_hat[i,i] = -sum(qmatrix_hat[i,-i])

print("estimate")
qmatrix_hat

print("true qmatrix")
qmatrix


[1] "estimate"


-0.7589751,0.1997485,0.1955350,0.3636916
0.8103100,-1.3868857,0.2395933,0.3369824
0.7386380,0.2231075,-1.3804264,0.4186809
0.7094355,0.1620389,0.2162000,-1.0876744


[1] "true qmatrix"


-0.7619048,0.1904762,0.1904762,0.3809524
0.7619048,-1.3333333,0.1904762,0.3809524
0.7619048,0.1904762,-1.3333333,0.3809524
0.7619048,0.1904762,0.1904762,-1.1428571


3.7 Estimating 6 rates

In [ ]:
qmatrix_hat = qmatrix_hat %*% diag(1/pi_hat)
qmatrix_hat = qmatrix_hat/qmatrix_hat[3,4]
rate_hat = qmatrix_hat[upper.tri(qmatrix_hat)]

print("rate estimates")
rate_hat

print("true rates")
rates

[1] "rate estimates"


[1] 0.9921752 0.9044172 1.1082021 0.8686605 0.8048670 1.0000000

[1] "true rates"


[1] 1 1 1 1 1 1